In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

from glob import glob
from datetime import datetime, timedelta

import matplotlib.pyplot as plt

In [ ]:
obdir = '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/observations/'

In [ ]:
site = 'CLN'
# f = glob(obdir + '%s*.csv'%site)[0]

#### Isolate precipitating (snow) periods

In [ ]:
# Write these out to a file later
# precip_periods = data_auto[(
#     (data_auto['snow_auto_mm'] > 0.) & 
#     (data_auto['swe_auto_mm'] > 0.) &
#     (data_auto['tsfc_c'] <= 6.))].index

# precip_periods3h = data_auto3h[(
#     (data_auto3h['snow_auto_mm'] > 0.) & 
#     (data_auto3h['swe_auto_mm'] > 0.) &
#     (data_auto3h['tsfc_c'] <= 6.))].index

# precip_periods[:5], precip_periods3h[:5]

#### Now the heavy lifting: extract a profile from the ERA5 and combine with the observation data
This will allow us to have our predictor set and target variable in one dataset to train a Machine Learning model

#### Determine ERA5 lat/lon gridpoint
Import the metadata file and take what's needed

In [ ]:
meta_file = glob(obdir + '*Metadata*.xlsx')[0]
metadata = pd.read_excel(meta_file).set_index('code').loc[site]

# Determine the start, end years from data if metadata is nan
start, end = metadata['start'], metadata['end']
start = data_save.index[0].year if np.isnan(start) else start
start = data_save.index[0].year if data_save.index[0].year < start else start
end = data_save.index[-1].year if np.isnan(end) else end
end = data_save.index[-1].year if data_save.index[-1].year > end else end
metadata['start'], metadata['end'] = start, end

# Determine the lat, lon of the site from the metadata
site_lat, site_lon = metadata['lat'], metadata['lon']
site_elev = metadata['elevation_m']

metadata

#### Check to see if a profile for this lat/lon exists

In [ ]:
# Import the era5_orog file to check the lat/lon grid
era_dir = '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/era5/'
era5_orog_file = era_dir + 'era5_orog.nc'
era5_orog = xr.open_dataset(era5_orog_file)['z'].isel(time=0)
era5_orog = era5_orog.rename({'latitude':'lat', 'longitude':'lon'})
era5_lat, era5_lon = era5_orog['lat'], era5_orog['lon']

# Find the index of the correct lat lon
idx1d = (np.abs(era5_lon - site_lon) + np.abs(era5_lat - site_lat))
idx = np.unravel_index(np.argmin(idx1d, axis=None), idx1d.shape)

# Subset and convert gpm to m
era5_g = 9.80665
era5_orog = era5_orog.isel(lat=idx[1], lon=idx[0])/era5_g

era5_orog

In [ ]:
# A string based on the lat and lon of the nearest era5 gridpoint
era5_prof_file = 'era5prof_{}N_{}W.nc'.format(
    era5_orog['lat'].values, abs(era5_orog['lon'].values))

print(era_dir + era5_prof_file)

if isfile(era_dir + 'profiles/' + era5_prof_file):
    # open profile
    era5_prof = xr.open_dataset(era_dir + 'profiles/' + era5_prof_file)
    
    # check start, end 
    # if start, end outside of bounds, produce the missing years
    # call the profile script with arguments for lat, lon, start, end
    # concatenate the variables for new years, save output
    # concatenate new years with old years, save output
    
else:
    print('no')
    # if profile is not found produce new
    # call the profile script with arguments for lat, lon, start, end
    # concatenate the variables, save output

## Task List:
#### Import profile data in the aggregate
#### Export the compiled data as netCDF
#### Now all training gridpoints/sites will be in a consistent format and easy to aggregate!
We can either expand the level variables here and calculate derived variables now or in a later script